In [ ]:
#different versio of numpy required for top2vec
!pip uninstall numpy
!pip install numpy==1.19.5

In [ ]:
#install the packages
# !pip install sentence-transformers
!pip install nltk
!pip install torch
!pip install tensorflow
!pip install tensorboard
!pip install top2vec
!pip install top2vec[sentence_encoders]
!pip install top2vec[sentence_transformers]
!pip install top2vec[indexing]

In [ ]:
#import required packages
import random
from tqdm import tqdm
import numpy as np
import os
from sentence_transformers import SentenceTransformer
import torch
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
!git clone https://github.com/akashjorss/sentence_bert
!unzip sentence_bert/sentences.zip
import pickle
with open('sentences.bin', 'rb') as f:
  sentences = pickle.load(f)

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
#load tensorboards with magics
%tensorflow_version 2.x
%load_ext tensorboard

In [12]:
documents = sentences#random.choices(sentences,k = 5000)
len(documents)

986637

In [ ]:
from top2vec import Top2Vec
model = Top2Vec(documents, embedding_model='universal-sentence-encoder', workers=4)

2021-05-06 06:59:09,021 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2021-05-06 07:00:05,621 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2021-05-06 07:00:10,359 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2021-05-06 07:03:06,030 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents


In [10]:
model.get_num_topics()

2

In [ ]:
topic_sizes, topic_nums = model.get_topic_sizes()
print("Topic sizes: ", topic_sizes)
print("Topic Numbers: ", topic_nums)

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["cleanliness"], num_topics=5)
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["staff"], num_topics=5)
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)